In [74]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import statsmodels.api as sm
import xgboost as xgb
from numpy import mean
from sklearn import preprocessing
from sklearn import svm as svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot


from sklearn.metrics import r2_score, roc_auc_score, accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

from sklearn.utils import resample
random.seed(123)

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
pd.set_option('display.max_columns', 500)


In [3]:
# import dataframe 

df = pd.read_csv('T2.csv', low_memory=False)

In [14]:
req_vars = ['SwellOneExit', 'YearsAsCoach', 'DaysTSubmitNoMissing',
       'FitNoMissing', 'RelationshipBuildingNoMissing',
       'FinalAchievementNoMissing', 'CommAndPresNoMissing',
       'PredModelScoreNoMissing', 'CMProspectRatingNoMissing',
       'AgeAtFDOSNoMissing', 'SchoolSelectivityNoMissing',
       'CumuGPANoMissing', 'PerseveranceNoMissing', 'FRPLNoMissing',
       'YearsSchoolPartnerNoMissing', 'YearsPartnerPartnerNoMissing',
       'FirstYearsInSchoolNoMissing', 'SecondYearsInSchoolNoMissing',
       'CMsInSchoolNoMissing', 'CertProgramCostNoMissing',
       'AvgFirstMonthSalNoMissing', 'RentPropNoMissing',
       'AvgRentNoMissing', 'CorpsSizeNoMissing', 'RegPrefLevNoMissing',
       'CSI6NoMissing', 'CSI5NoMissing', 'CSI3NoMissing',
       'CSI12NoMissing', 'CLI8NoMissing', 'CLI6NoMissing',
       'OtherCMsSameCoachNoMissing', 'OtherCMsPrevCoachNoMissing',
       'K12TeachNo', 'K12TeachYes', 'K12TeachNull', 'HadFamRespNo',
       'HadFamRespYes', 'HadFamRespNull', 'PellGrantNoOrMissing',
       'PellGrantYes', 'GenderFemale', 'GenderMale', 'CalcGradStudent',
       'CalcJunior', 'CalcProfessional', 'CalcUndergrad', 'CalcUnknown',
       'AttendLIC_HS_No', 'AttendLIC_HS_Yes', 'AttendLIC_HS_Unknown',
       'GrewUpLIC_No_Or_Unknown', 'GrewUpLIC_Yes',
       'LIC_Served_No_Or_Unknown', 'LIC_Served_Yes', 'Eth_Black',
       'Eth_Native', 'Eth_AsianAmPacIsl', 'Eth_Latinx',
       'Eth_MultiEthMultiRacial', 'Eth_OtherOrUnknown', 'Eth_White',
       'VarsitySport_No_Or_Unknown', 'VarsitySport_Yes',
       'DeferralRequested', 'DeferralNotRequested', 'Title1No',
       'Title1Yes', 'Grade_ECE', 'Grade_HIGH', 'Grade_LOWELEM',
       'Grade_MIDDLE', 'Grade_UPPERELEM', 'Grade_Unknown',
       'School_Charter', 'School_Public', 'School_Other', 'LGB_NoProtect',
       'LGB_LocalProtect', 'LGB_StateProtect', 'TeacherPP_No',
       'TeacherPP_Yes', 'Region_Large', 'Region_Medium', 'Region_Small',
       'Urbanicity_Both', 'Urbanicity_Rural', 'Urbanicity_Urban',
       'COL_High', 'COL_Low', 'COL_Medium', 'MajorTeaching_Match',
       'MajorTeaching_Mismatch', 'MajorOffered_Match',
       'MajorOffered_Mismatch']

df_req_vars = df[req_vars] # updating the dataframe with only required variables

Y = df_req_vars['SwellOneExit']
X = df_req_vars.drop('SwellOneExit', axis=1)

In [33]:
# Cardinality Test

vars_cardinality = {}

for var in df_req_vars.columns:
    vars_cardinality[var] = df[var].nunique() # prints the # of unique values the variable has

In [55]:
vars_cardinality

{'SwellOneExit': 2,
 'YearsAsCoach': 13,
 'DaysTSubmitNoMissing': 213,
 'FitNoMissing': 10,
 'RelationshipBuildingNoMissing': 27,
 'FinalAchievementNoMissing': 207,
 'CommAndPresNoMissing': 11,
 'PredModelScoreNoMissing': 19519,
 'CMProspectRatingNoMissing': 12,
 'AgeAtFDOSNoMissing': 2212,
 'SchoolSelectivityNoMissing': 6,
 'CumuGPANoMissing': 165,
 'PerseveranceNoMissing': 10,
 'FRPLNoMissing': 99,
 'YearsSchoolPartnerNoMissing': 13,
 'YearsPartnerPartnerNoMissing': 14,
 'FirstYearsInSchoolNoMissing': 17,
 'SecondYearsInSchoolNoMissing': 17,
 'CMsInSchoolNoMissing': 27,
 'CertProgramCostNoMissing': 46,
 'AvgFirstMonthSalNoMissing': 35,
 'RentPropNoMissing': 50,
 'AvgRentNoMissing': 35,
 'CorpsSizeNoMissing': 187,
 'RegPrefLevNoMissing': 4,
 'CSI6NoMissing': 8,
 'CSI5NoMissing': 8,
 'CSI3NoMissing': 8,
 'CSI12NoMissing': 8,
 'CLI8NoMissing': 8,
 'CLI6NoMissing': 8,
 'OtherCMsSameCoachNoMissing': 54,
 'OtherCMsPrevCoachNoMissing': 133,
 'K12TeachNo': 2,
 'K12TeachYes': 2,
 'K12TeachNul

In [57]:
cont_vars = [] # making a list of continuous variables

for key in vars_cardinality:
    if vars_cardinality[key] > 10: # setting 10 as the threshold
        cont_vars.append(key) # list of continuous variables

In [66]:
df_cont_vars = df_req_vars[cont_vars] # dataframe of continuous variables

df_cat_vars = df_req_vars.drop(cont_vars, axis=1) # dataframe of categorical variables

df_cat_vars.sample(5)

,SwellOneExit,FitNoMissing,SchoolSelectivityNoMissing,PerseveranceNoMissing,RegPrefLevNoMissing,CSI6NoMissing,CSI5NoMissing,CSI3NoMissing,CSI12NoMissing,CLI8NoMissing,...,Urbanicity_Both,Urbanicity_Rural,Urbanicity_Urban,COL_High,COL_Low,COL_Medium,MajorTeaching_Match,MajorTeaching_Mismatch,MajorOffered_Match,MajorOffered_Mismatch
21220,0,4.0,4.0,2.5,1.0,5.000000,6.000000,6.000000,6.000000,7.000000,...,0,0,1,0,0,1,0,1,0,1
18469,0,4.0,3.0,3.5,1.0,7.000000,5.000000,7.000000,3.000000,4.000000,...,0,0,1,0,1,0,0,1,0,1
7027,0,4.0,4.0,4.0,2.0,5.915762,5.804889,5.754843,5.658548,6.185857,...,0,0,1,1,0,0,0,1,1,0
39,0,5.0,2.0,4.5,1.0,4.000000,6.000000,4.000000,6.000000,7.000000,...,0,0,1,0,1,0,0,1,0,1
2150,0,4.0,2.0,3.0,1.0,5.000000,4.000000,5.000000,6.000000,5.000000,...,0,0,1,1,0,0,0,1,1,0


In [69]:
def normalize(df):
    df_cont_vars_normalized = df_cont_vars.copy()
    for feature_name in df_cont_vars_normalized.columns:
        mean = df[feature_name].mean()
        std = df[feature_name].std()
        df_cont_vars_normalized[feature_name] = (df_cont_vars_normalized[feature_name] - mean) / std
    return df_cont_vars_normalized

df_cont_vars_scaled = normalize(df_cont_vars)

df_cont_vars_scaled.sample(5)

,YearsAsCoach,DaysTSubmitNoMissing,RelationshipBuildingNoMissing,FinalAchievementNoMissing,CommAndPresNoMissing,PredModelScoreNoMissing,CMProspectRatingNoMissing,AgeAtFDOSNoMissing,CumuGPANoMissing,FRPLNoMissing,...,FirstYearsInSchoolNoMissing,SecondYearsInSchoolNoMissing,CMsInSchoolNoMissing,CertProgramCostNoMissing,AvgFirstMonthSalNoMissing,RentPropNoMissing,AvgRentNoMissing,CorpsSizeNoMissing,OtherCMsSameCoachNoMissing,OtherCMsPrevCoachNoMissing
9363,-0.083565,-0.160559,1.354639e+00,-0.800321,-0.481461,-0.790357,-1.804725,-0.212054,-0.383940,-6.470770e-08,...,1.606901e-08,-0.777610,-1.327035e-08,1.491018,0.279141,-0.277358,-0.142010,-0.409641,0.609513,0.121057
5600,0.637842,0.692513,4.650780e-01,-0.593865,1.334516,-0.227920,-1.420761,-0.195304,-0.501589,-6.470770e-08,...,4.988316e-01,-0.777610,-1.784390e-01,-1.364961,-0.077984,-0.752014,-0.739294,0.303131,-0.674576,0.891436
4161,0.637842,-0.099625,-1.338087e+00,1.087274,0.117811,-0.968026,-1.420761,-0.474480,1.098444,2.612569e-01,...,1.487217e+00,1.688296,1.865764e+00,-0.279038,-0.345827,-0.189138,-0.317682,0.281748,-0.460561,1.527836
16665,-0.804973,0.052709,-5.086091e-08,0.556388,-0.481461,-1.987717,-1.420761,0.608724,-0.431000,-1.374176e+00,...,4.638855e-03,-0.284429,-1.784390e-01,0.263924,-2.533214,3.606481,0.947154,-0.516557,0.181483,-0.883784
11332,-0.083565,0.692513,7.535844e-01,-1.714624,0.735243,0.026867,-1.420761,-0.143191,-0.713358,6.885308e-02,...,-4.895539e-01,-0.777610,-7.624971e-01,-1.328764,-1.452912,-1.263952,-1.582518,-0.224320,-0.567568,-0.079911


In [72]:
df_final = pd.concat([df_cont_vars_scaled, df_cat_vars], axis=1)

df_final.sample(5)

(22992, 95)

In [61]:
preproc_cont_vars = preprocessing.scale(df_cont_vars)

pd.DataFrame(preproc_cont_vars)

sklearn.preprocessing.StandardScaler

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.000026,-0.678510,0.753601,-1.537696,0.735259,0.807082,-0.268875,3.231189,-1.019269,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,-1.349607,-1.149382,-1.408639,-1.617688,-1.471703,-0.000001,-2.824940e-07
1,-0.000026,0.997203,1.955737,-0.859327,0.735259,0.969325,-1.420792,2.741690,-1.019269,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,0.553516,0.368430,-0.244253,-0.071743,-0.438161,-0.000001,-2.824940e-07
2,-0.000026,0.814398,-0.448535,-0.033486,0.735259,-0.025789,-0.268875,-0.303259,0.157251,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,0.877490,0.457713,2.205895,2.493119,0.566869,-0.000001,-2.824940e-07
3,-0.000026,-0.587108,1.955737,0.054997,0.117814,-0.209258,0.499069,-0.221365,0.063130,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,0.877490,0.457713,2.205895,2.493119,0.566869,-0.000001,-2.824940e-07
4,-0.000026,-0.526173,1.955737,0.703872,0.735259,0.901445,-0.268875,-0.145055,0.439616,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,-1.070880,0.857701,-0.783639,-0.458229,-0.138790,-0.000001,-2.824940e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22987,-0.804990,-0.769913,-1.650671,-0.357923,-1.080758,-1.883212,0.499069,-0.413070,-0.054522,-3.159614e-01,...,-4.895646e-01,-0.777626,-7.625137e-01,1.440373,1.112461,-0.485819,-0.036608,-1.257867,-0.139542,-8.838035e-01
22988,-0.804990,0.601125,-0.448535,1.205275,0.735259,1.813890,0.499069,-0.195308,0.886694,6.885458e-02,...,-9.837681e-01,-0.777626,-1.054549e+00,1.440373,1.112461,-0.485819,-0.036608,-1.257867,-0.139542,-8.838035e-01
22989,-0.804990,-0.769913,-1.650671,1.116792,0.735259,1.322207,0.499069,4.414922,-0.030992,3.093646e-01,...,-4.895646e-01,0.208757,-1.784429e-01,1.440373,1.112461,-0.485819,-0.036608,-1.257867,-0.139542,-8.838035e-01
22990,-0.804990,-0.373835,-1.650671,0.113985,-0.481472,0.710420,0.499069,-0.385152,0.227843,-7.007774e-01,...,-4.895646e-01,-0.777626,-7.625137e-01,1.440373,1.112461,-0.485819,-0.036608,-1.257867,-0.139542,-8.838035e-01


In [42]:
pd.DataFrame(preproc_cont_vars)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.000026,-0.678510,0.753601,-1.537696,0.735259,0.807082,-0.268875,3.231189,-1.019269,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,-1.349607,-1.149382,-1.408639,-1.617688,-1.471703,-0.000001,-2.824940e-07
1,-0.000026,0.997203,1.955737,-0.859327,0.735259,0.969325,-1.420792,2.741690,-1.019269,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,0.553516,0.368430,-0.244253,-0.071743,-0.438161,-0.000001,-2.824940e-07
2,-0.000026,0.814398,-0.448535,-0.033486,0.735259,-0.025789,-0.268875,-0.303259,0.157251,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,0.877490,0.457713,2.205895,2.493119,0.566869,-0.000001,-2.824940e-07
3,-0.000026,-0.587108,1.955737,0.054997,0.117814,-0.209258,0.499069,-0.221365,0.063130,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,0.877490,0.457713,2.205895,2.493119,0.566869,-0.000001,-2.824940e-07
4,-0.000026,-0.526173,1.955737,0.703872,0.735259,0.901445,-0.268875,-0.145055,0.439616,-6.470924e-08,...,1.606935e-08,-0.777626,-1.327064e-08,-1.070880,0.857701,-0.783639,-0.458229,-0.138790,-0.000001,-2.824940e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22987,-0.804990,-0.769913,-1.650671,-0.357923,-1.080758,-1.883212,0.499069,-0.413070,-0.054522,-3.159614e-01,...,-4.895646e-01,-0.777626,-7.625137e-01,1.440373,1.112461,-0.485819,-0.036608,-1.257867,-0.139542,-8.838035e-01
22988,-0.804990,0.601125,-0.448535,1.205275,0.735259,1.813890,0.499069,-0.195308,0.886694,6.885458e-02,...,-9.837681e-01,-0.777626,-1.054549e+00,1.440373,1.112461,-0.485819,-0.036608,-1.257867,-0.139542,-8.838035e-01
22989,-0.804990,-0.769913,-1.650671,1.116792,0.735259,1.322207,0.499069,4.414922,-0.030992,3.093646e-01,...,-4.895646e-01,0.208757,-1.784429e-01,1.440373,1.112461,-0.485819,-0.036608,-1.257867,-0.139542,-8.838035e-01
22990,-0.804990,-0.373835,-1.650671,0.113985,-0.481472,0.710420,0.499069,-0.385152,0.227843,-7.007774e-01,...,-4.895646e-01,-0.777626,-7.625137e-01,1.440373,1.112461,-0.485819,-0.036608,-1.257867,-0.139542,-8.838035e-01


In [25]:
df_req_vars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22992 entries, 0 to 22991
Data columns (total 95 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SwellOneExit                   22992 non-null  int64  
 1   YearsAsCoach                   22992 non-null  float64
 2   DaysTSubmitNoMissing           22992 non-null  float64
 3   FitNoMissing                   22992 non-null  float64
 4   RelationshipBuildingNoMissing  22992 non-null  float64
 5   FinalAchievementNoMissing      22992 non-null  float64
 6   CommAndPresNoMissing           22992 non-null  float64
 7   PredModelScoreNoMissing        22992 non-null  float64
 8   CMProspectRatingNoMissing      22992 non-null  float64
 9   AgeAtFDOSNoMissing             22992 non-null  float64
 10  SchoolSelectivityNoMissing     22992 non-null  float64
 11  CumuGPANoMissing               22992 non-null  float64
 12  PerseveranceNoMissing          22992 non-null 

In [23]:
normalized_X = preprocessing.normalize(X)

standardized_X = preprocessing.scale(X)

In [62]:
pd.DataFrame(normalized_X)

,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
0,0.000380,0.001022,0.001533,0.001363,0.001026,0.001479,0.000263,0.001704,0.014342,0.001022,...,0.000000,0.000341,0.000000,0.000000,0.000341,0.000000,0.0,0.000341,0.000000,0.000341
1,0.000099,0.005138,0.000354,0.000399,0.000287,0.000384,0.000069,0.000177,0.003496,0.000177,...,0.000000,0.000000,0.000089,0.000089,0.000000,0.000000,0.0,0.000089,0.000000,0.000089
2,0.000085,0.003969,0.000305,0.000267,0.000269,0.000331,0.000054,0.000382,0.001763,0.000229,...,0.000000,0.000000,0.000076,0.000076,0.000000,0.000000,0.0,0.000076,0.000000,0.000076
3,0.000085,0.000458,0.000191,0.000343,0.000271,0.000305,0.000053,0.000534,0.001797,0.000229,...,0.000000,0.000000,0.000076,0.000076,0.000000,0.000000,0.0,0.000076,0.000000,0.000076
4,0.000252,0.001807,0.000903,0.001016,0.000852,0.000980,0.000176,0.001129,0.005410,0.000226,...,0.000226,0.000000,0.000000,0.000000,0.000000,0.000226,0.0,0.000226,0.000000,0.000226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22987,0.000000,0.000000,0.000124,0.000187,0.000212,0.000208,0.000036,0.000435,0.001400,0.000187,...,0.000062,0.000000,0.000000,0.000062,0.000000,0.000000,0.0,0.000062,0.000062,0.000000
22988,0.000000,0.002798,0.000311,0.000218,0.000245,0.000270,0.000053,0.000435,0.001472,0.000187,...,0.000062,0.000000,0.000000,0.000062,0.000000,0.000000,0.0,0.000062,0.000062,0.000000
22989,0.000000,0.000000,0.000249,0.000187,0.000243,0.000270,0.000050,0.000435,0.003013,0.000187,...,0.000062,0.000000,0.000000,0.000062,0.000000,0.000000,0.0,0.000062,0.000062,0.000000
22990,0.000000,0.000808,0.000311,0.000187,0.000222,0.000228,0.000048,0.000435,0.001409,0.000187,...,0.000062,0.000000,0.000000,0.000062,0.000000,0.000000,0.0,0.000062,0.000062,0.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22992 entries, 0 to 22991
Data columns (total 95 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SwellOneExit                   22992 non-null  int64  
 1   YearsAsCoach                   22992 non-null  float64
 2   DaysTSubmitNoMissing           22992 non-null  float64
 3   FitNoMissing                   22992 non-null  float64
 4   RelationshipBuildingNoMissing  22992 non-null  float64
 5   FinalAchievementNoMissing      22992 non-null  float64
 6   CommAndPresNoMissing           22992 non-null  float64
 7   PredModelScoreNoMissing        22992 non-null  float64
 8   CMProspectRatingNoMissing      22992 non-null  float64
 9   AgeAtFDOSNoMissing             22992 non-null  float64
 10  SchoolSelectivityNoMissing     22992 non-null  float64
 11  CumuGPANoMissing               22992 non-null  float64
 12  PerseveranceNoMissing          22992 non-null 

In [9]:
df_req_vars.describe()

,SwellOneExit,YearsAsCoach,DaysTSubmitNoMissing,FitNoMissing,RelationshipBuildingNoMissing,FinalAchievementNoMissing,CommAndPresNoMissing,PredModelScoreNoMissing,CMProspectRatingNoMissing,AgeAtFDOSNoMissing,...,Urbanicity_Both,Urbanicity_Rural,Urbanicity_Urban,COL_High,COL_Low,COL_Medium,MajorTeaching_Match,MajorTeaching_Mismatch,MajorOffered_Match,MajorOffered_Mismatch
count,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,...,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000,22992.000000
mean,0.038970,1.115836,25.269958,3.756810,3.686558,3.531353,3.935125,0.712088,5.700246,24.729358,...,0.203158,0.055150,0.741693,0.367563,0.292841,0.339596,0.305498,0.694502,0.600818,0.399182
std,0.193528,1.386179,32.822559,0.884409,0.415935,0.339056,0.550668,0.073395,2.604412,5.372950,...,0.402358,0.228277,0.437713,0.482152,0.455076,0.473583,0.460628,0.460628,0.489741,0.489741
min,0.000000,0.000000,0.000000,1.000000,1.000000,1.600000,2.000000,0.439480,0.000000,0.280000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,3.000000,3.000000,3.500000,3.300000,3.670000,0.662917,4.000000,22.340000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,14.000000,4.000000,3.686558,3.580000,4.000000,0.705444,5.700246,22.890000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,0.000000,2.000000,34.000000,4.500000,4.000000,3.800000,4.340000,0.757622,8.000000,24.660000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,11.000000,217.000000,5.000000,5.000000,4.250000,5.000000,1.008109,10.000000,74.980000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [76]:
df_final.to_csv('Dataset_Scaled.csv')